In [1]:
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import numpy as np
import argparse
import imutils
import cv2

Using TensorFlow backend.
/home/autom8-sudesh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/autom8-sudesh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/autom8-sudesh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/autom8-sudesh/.local/lib/python3.6/sit

In [2]:
detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
model = load_model('model.hdf5')
EMOTIONS = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [3]:
camera = cv2.VideoCapture(0)



In [4]:
while True:
    # grab the current frame
    (grabbed, frame) = camera.read()
    frame = imutils.resize(frame, width=300)
    print(frame)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    canvas = np.zeros((220, 300, 3), dtype="uint8")
    frameClone = frame.copy()
    rects = detector.detectMultiScale(gray, scaleFactor=1.1,minNeighbors=5, minSize=(30, 30),flags=cv2.CASCADE_SCALE_IMAGE)
    if len(rects) > 0:
        # determine the largest face area
        rect = sorted(rects, reverse=True,
        key=lambda x: (x[2] - x[0]) * (x[3] - x[1]))[0]
        (fX, fY, fW, fH) = rect
        roi = gray[fY:fY + fH, fX:fX + fW]
        roi = cv2.resize(roi, (48, 48))
        roi = roi.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)
        preds = model.predict(roi)[0]
        label = EMOTIONS[preds.argmax()]
        for (i, (emotion, prob)) in enumerate(zip(EMOTIONS, preds)):
            # construct the label text
            text = "{}: {:.2f}%".format(emotion, prob * 100)
            w = int(prob * 300)
            cv2.rectangle(canvas, (5, (i * 35) + 5),
            (w, (i * 35) + 35), (0, 0, 255), -1)
            cv2.putText(canvas, text, (10, (i * 35) + 23),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45,
            (255, 255, 255), 2)
            
        cv2.putText(frameClone, label, (fX, fY - 10),
        cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
        cv2.rectangle(frameClone, (fX, fY), (fX + fW, fY + fH),
        (0, 0, 255), 2)
    cv2.imshow("Face", frameClone)
    cv2.imshow("Probabilities", canvas)
    if cv2.waitKey(1) & 0xFF == ord("q"):
            break
camera.release()
cv2.destroyAllWindows()

[[[207 177 182]
  [203 173 176]
  [199 175 176]
  ...
  [ 54  54  50]
  [ 39  48  45]
  [ 32  40  33]]

 [[198 172 178]
  [198 174 176]
  [199 177 176]
  ...
  [ 46  44  26]
  [ 43  42  32]
  [ 44  40  40]]

 [[203 170 185]
  [203 173 179]
  [207 181 182]
  ...
  [ 44  46  35]
  [ 45  48  41]
  [ 43  45  41]]

 ...

 [[ 74  52  60]
  [ 70  53  64]
  [ 57  54  66]
  ...
  [ 98  85 108]
  [ 90  77 107]
  [ 66  53  88]]

 [[ 70  49  54]
  [ 76  66  70]
  [ 63  65  73]
  ...
  [107 102 109]
  [101 105 112]
  [ 86 101 108]]

 [[ 72  54  79]
  [ 76  67  83]
  [ 57  58  69]
  ...
  [120 123 115]
  [106 114 103]
  [105 118 106]]]
[[[202 177 181]
  [199 170 173]
  [205 174 173]
  ...
  [ 34  58  52]
  [ 38  45  43]
  [ 42  32  33]]

 [[196 180 164]
  [198 180 169]
  [206 182 178]
  ...
  [ 33  54  36]
  [ 44  56  43]
  [ 51  55  47]]

 [[201 175 163]
  [200 175 162]
  [204 178 173]
  ...
  [ 38  50  37]
  [ 42  49  29]
  [ 53  58  34]]

 ...

 [[ 50  60  72]
  [ 56  61  70]
  [ 56  57  61]
  ..

[[[206 182 174]
  [207 176 171]
  [214 174 176]
  ...
  [ 52  41  46]
  [ 63  39  46]
  [ 71  34  39]]

 [[199 186 172]
  [198 177 171]
  [196 174 174]
  ...
  [ 50  46  27]
  [ 60  37  32]
  [ 70  30  36]]

 [[192 176 174]
  [194 175 174]
  [200 183 181]
  ...
  [ 41  47  31]
  [ 50  41  32]
  [ 51  28  25]]

 ...

 [[ 62  54  60]
  [ 52  45  50]
  [ 64  59  65]
  ...
  [ 86  82 105]
  [ 85  75 107]
  [ 72  57  91]]

 [[ 65  49  61]
  [ 67  49  62]
  [ 80  64  73]
  ...
  [ 95  91  98]
  [102  91 104]
  [ 97  80  99]]

 [[ 73  54  67]
  [ 76  57  66]
  [ 75  58  61]
  ...
  [115 116 111]
  [114 108 108]
  [112 102 104]]]
[[[198 179 174]
  [197 175 172]
  [204 182 178]
  ...
  [ 48  37  52]
  [ 52  38  39]
  [ 54  40  31]]

 [[208 178 174]
  [203 171 169]
  [208 174 180]
  ...
  [ 44  48  33]
  [ 41  38  21]
  [ 58  42  27]]

 [[204 174 175]
  [201 171 172]
  [209 181 183]
  ...
  [ 41  46  37]
  [ 41  41  29]
  [ 45  38  27]]

 ...

 [[ 57  65  83]
  [ 57  62  74]
  [ 55  58  70]
  ..

[[[203 186 175]
  [196 182 176]
  [198 185 186]
  ...
  [ 58  39  43]
  [ 51  42  48]
  [ 43  42  53]]

 [[200 183 178]
  [196 180 178]
  [202 182 186]
  ...
  [ 43  36  35]
  [ 53  43  42]
  [ 53  43  45]]

 [[195 176 179]
  [195 175 178]
  [199 177 180]
  ...
  [ 36  42  31]
  [ 47  38  33]
  [ 54  39  37]]

 ...

 [[ 51  63  57]
  [ 51  62  54]
  [ 61  65  62]
  ...
  [ 86  88  97]
  [ 90  90 100]
  [ 69  71  85]]

 [[ 59  57  53]
  [ 60  64  59]
  [ 55  65  62]
  ...
  [104 108 111]
  [ 96 101 102]
  [ 82  86  87]]

 [[ 61  50  64]
  [ 61  64  71]
  [ 49  69  66]
  ...
  [107 116 107]
  [105 119 112]
  [102 116 107]]]
[[[202 184 171]
  [196 175 167]
  [199 175 178]
  ...
  [ 54  41  39]
  [ 45  37  43]
  [ 44  41  56]]

 [[198 181 159]
  [196 179 162]
  [206 185 183]
  ...
  [ 47  43  36]
  [ 50  40  40]
  [ 55  44  51]]

 [[198 176 175]
  [196 175 171]
  [199 180 174]
  ...
  [ 45  46  49]
  [ 50  41  47]
  [ 60  45  51]]

 ...

 [[ 43  61  73]
  [ 53  57  70]
  [ 55  51  63]
  ..

[[[197 181 164]
  [199 183 170]
  [199 176 178]
  ...
  [ 51  35  37]
  [ 50  37  44]
  [ 45  38  46]]

 [[203 183 170]
  [200 178 170]
  [197 178 178]
  ...
  [ 46  54  58]
  [ 38  47  53]
  [ 42  48  53]]

 [[202 168 167]
  [204 176 170]
  [203 182 178]
  ...
  [ 56  50  60]
  [ 56  47  55]
  [ 59  47  52]]

 ...

 [[ 64  62  77]
  [ 57  59  68]
  [ 51  59  62]
  ...
  [130 115 120]
  [127 113 113]
  [123 107 103]]

 [[ 49  62  73]
  [ 50  65  71]
  [ 53  63  74]
  ...
  [134 126 128]
  [130 123 124]
  [128 124 124]]

 [[ 41  65  75]
  [ 46  64  71]
  [ 44  57  61]
  ...
  [137 123 124]
  [116 114 109]
  [119 128 114]]]
[[[215 182 158]
  [210 175 161]
  [206 173 171]
  ...
  [ 44  48  46]
  [ 47  48  37]
  [ 44  46  24]]

 [[196 185 170]
  [195 184 171]
  [191 179 172]
  ...
  [ 49  44  32]
  [ 53  43  28]
  [ 59  42  33]]

 [[192 187 172]
  [185 180 165]
  [187 183 173]
  ...
  [ 44  47  40]
  [ 49  46  38]
  [ 55  49  46]]

 ...

 [[ 61  65  93]
  [ 52  61  80]
  [ 51  64  76]
  ..

[[[200 169 160]
  [213 185 178]
  [206 185 176]
  ...
  [ 38  43  27]
  [ 50  55  41]
  [ 45  51  38]]

 [[205 169 172]
  [209 172 174]
  [208 176 179]
  ...
  [ 43  55  38]
  [ 39  59  45]
  [ 32  55  46]]

 [[204 172 171]
  [202 169 170]
  [202 172 177]
  ...
  [ 50  50  36]
  [ 49  45  43]
  [ 51  49  49]]

 ...

 [[ 60  57  58]
  [ 63  53  56]
  [ 68  54  62]
  ...
  [126 122 128]
  [123 123 128]
  [111 118 119]]

 [[ 65  59  48]
  [ 64  53  48]
  [ 74  53  57]
  ...
  [128 126 133]
  [123 118 125]
  [123 114 126]]

 [[ 63  59  50]
  [ 69  59  57]
  [ 74  56  66]
  ...
  [129 129 130]
  [122 121 119]
  [121 118 113]]]
[[[200 184 187]
  [196 181 181]
  [197 176 177]
  ...
  [ 61  45  39]
  [ 69  42  30]
  [ 70  32  13]]

 [[201 180 183]
  [196 179 179]
  [199 184 180]
  ...
  [ 54  47  33]
  [ 45  45  23]
  [ 51  49  21]]

 [[196 176 178]
  [194 178 178]
  [200 185 183]
  ...
  [ 52  44  39]
  [ 55  42  35]
  [ 61  47  36]]

 ...

 [[ 68  60  58]
  [ 64  59  57]
  [ 62  60  57]
  ..

...

[[[217 178 182]
  [206 165 172]
  [214 175 181]
  ...
  [ 44  38  57]
  [ 49  39  52]
  [ 48  38  41]]

 [[211 181 177]
  [196 169 167]
  [200 176 178]
  ...
  [ 52  52  62]
  [ 41  37  47]
  [ 46  40  44]]

 [[201 182 178]
  [193 178 172]
  [194 179 177]
  ...
  [ 51  55  56]
  [ 47  51  47]
  [ 44  48  38]]

 ...

 [[ 63  72  83]
  [ 57  59  69]
  [ 68  66  73]
  ...
  [ 83  66 110]
  [ 84  66 118]
  [ 80  59 112]]

 [[ 66  57  71]
  [ 64  57  71]
  [ 69  60  76]
  ...
  [ 85  66 112]
  [ 90  59 121]
  [ 92  52 121]]

 [[ 78  52  77]
  [ 80  56  78]
  [ 76  53  77]
  ...
  [ 95  67 111]
  [ 93  56 109]
  [100  59 116]]]
[[[208 177 182]
  [206 176 181]
  [210 176 185]
  ...
  [ 54  52  51]
  [ 55  51  49]
  [ 53  46  42]]

 [[204 177 187]
  [200 179 183]
  [191 177 177]
  ...
  [ 52  52  51]
  [ 43  43  41]
  [ 44  46  41]]

 [[204 179 183]
  [197 178 179]
  [190 178 178]
  ...
  [ 42  55  49]
  [ 38  52  46]
  [ 39  57  49]]

 ...

 [[ 60  70  64]
  [ 64  68  63]
  [ 65  62  61]
  ..

[[[199 177 191]
  [193 172 175]
  [206 183 184]
  ...
  [ 69  38  32]
  [ 75  40  31]
  [ 66  32  27]]

 [[200 184 178]
  [202 181 174]
  [204 179 176]
  ...
  [ 55  47  36]
  [ 54  44  31]
  [ 60  54  43]]

 [[201 177 173]
  [201 173 171]
  [202 172 171]
  ...
  [ 49  55  55]
  [ 45  48  47]
  [ 50  56  54]]

 ...

 [[ 67  65  61]
  [ 70  66  67]
  [ 73  60  75]
  ...
  [ 77  67 108]
  [ 82  69 116]
  [ 69  55 104]]

 [[ 67  58  61]
  [ 72  62  68]
  [ 66  53  70]
  ...
  [ 76  60 105]
  [ 83  63 116]
  [ 81  61 118]]

 [[ 68  57  75]
  [ 73  59  76]
  [ 62  49  64]
  ...
  [ 74  62 111]
  [ 79  63 116]
  [ 86  70 124]]]
[[[213 189 176]
  [210 184 176]
  [205 180 173]
  ...
  [ 75  36  46]
  [ 70  41  51]
  [ 56  30  53]]

 [[206 186 174]
  [199 177 164]
  [202 179 165]
  ...
  [ 45  39  37]
  [ 51  41  41]
  [ 54  43  46]]

 [[198 182 176]
  [195 174 172]
  [201 174 175]
  ...
  [ 42  53  51]
  [ 42  48  45]
  [ 47  51  44]]

 ...

 [[ 36  73  72]
  [ 43  67  72]
  [ 56  63  76]
  ..

[[[202 187 162]
  [204 185 169]
  [203 183 181]
  ...
  [ 50  33  50]
  [ 51  25  36]
  [ 67  31  34]]

 [[204 181 165]
  [200 177 168]
  [198 180 175]
  ...
  [ 50  51  47]
  [ 49  36  35]
  [ 58  31  33]]

 [[203 179 182]
  [194 174 172]
  [195 179 172]
  ...
  [ 47  47  44]
  [ 49  36  30]
  [ 59  42  33]]

 ...

 [[ 67  62  70]
  [ 65  61  71]
  [ 56  57  65]
  ...
  [ 79  70 102]
  [ 86  75 109]
  [ 72  62  93]]

 [[ 77  50  86]
  [ 68  53  83]
  [ 67  59  82]
  ...
  [ 80  78 102]
  [ 80  81 104]
  [ 67  70  91]]

 [[ 69  56  79]
  [ 69  57  80]
  [ 60  53  68]
  ...
  [ 76  73 105]
  [ 74  75  97]
  [ 74  77  92]]]
[[[210 182 185]
  [204 178 181]
  [195 171 177]
  ...
  [ 55  39  41]
  [ 50  45  44]
  [ 46  44  56]]

 [[194 167 167]
  [202 177 178]
  [200 174 176]
  ...
  [ 64  43  44]
  [ 52  41  35]
  [ 55  48  41]]

 [[199 174 168]
  [198 172 169]
  [202 174 175]
  ...
  [ 49  43  35]
  [ 50  41  36]
  [ 62  54  48]]

 ...

 [[ 60  58  46]
  [ 62  60  57]
  [ 59  56  67]
  ..

[[[206 184 185]
  [200 180 177]
  [195 178 170]
  ...
  [ 61  40  49]
  [ 58  42  43]
  [ 60  44  32]]

 [[203 183 182]
  [200 181 179]
  [199 181 173]
  ...
  [ 46  39  42]
  [ 54  43  36]
  [ 53  45  29]]

 [[193 181 176]
  [194 181 174]
  [193 180 172]
  ...
  [ 40  37  39]
  [ 41  37  26]
  [ 48  53  33]]

 ...

 [[ 60  66  66]
  [ 56  61  67]
  [ 54  58  68]
  ...
  [ 66  76 101]
  [ 77  78 102]
  [ 76  66  86]]

 [[ 57  59  54]
  [ 62  66  66]
  [ 58  71  77]
  ...
  [ 71  72  98]
  [ 77  72  95]
  [ 79  72  93]]

 [[ 72  53  84]
  [ 65  53  77]
  [ 55  56  70]
  ...
  [ 72  74 106]
  [ 73  71  96]
  [ 75  73  89]]]
[[[217 181 168]
  [207 175 162]
  [209 183 172]
  ...
  [ 54  42  47]
  [ 62  40  57]
  [ 61  28  61]]

 [[192 172 173]
  [197 175 172]
  [202 180 173]
  ...
  [ 48  48  49]
  [ 56  40  53]
  [ 59  29  52]]

 [[180 181 174]
  [181 176 169]
  [195 183 177]
  ...
  [ 44  52  48]
  [ 44  51  47]
  [ 37  41  40]]

 ...

 [[ 60  55  68]
  [ 55  53  70]
  [ 54  60  76]
  ..

[[[210 185 160]
  [208 176 161]
  [206 174 172]
  ...
  [ 64  36  53]
  [ 56  23  47]
  [ 61  28  57]]

 [[204 181 170]
  [197 174 168]
  [198 177 180]
  ...
  [ 46  41  42]
  [ 49  37  39]
  [ 58  42  46]]

 [[201 175 180]
  [199 174 179]
  [200 177 186]
  ...
  [ 48  48  47]
  [ 46  38  41]
  [ 52  45  47]]

 ...

 [[ 64  61  62]
  [ 59  56  59]
  [ 55  55  63]
  ...
  [ 81  75 108]
  [ 84  75 106]
  [ 74  68  92]]

 [[ 54  63  64]
  [ 57  68  71]
  [ 54  67  77]
  ...
  [ 65  63  98]
  [ 76  68 103]
  [ 71  62  93]]

 [[ 51  73  89]
  [ 48  64  82]
  [ 52  61  83]
  ...
  [ 66  68 108]
  [ 74  65 102]
  [ 79  67 102]]]
[[[212 175 176]
  [210 175 175]
  [205 175 176]
  ...
  [ 48  44  34]
  [ 54  52  39]
  [ 47  50  44]]

 [[205 183 165]
  [207 180 167]
  [207 179 175]
  ...
  [ 46  52  43]
  [ 48  48  35]
  [ 51  45  34]]

 [[192 184 169]
  [191 178 169]
  [195 177 175]
  ...
  [ 48  52  41]
  [ 43  47  37]
  [ 45  48  39]]

 ...

 [[ 53  63  84]
  [ 56  65  81]
  [ 51  63  75]
  ..